# Unit 2

## Changeset Management API in FastAPI

## Implementing a Simple Health Check Endpoint

## Implementing Basic Changeset Listing Endpoint

## Adding Filtering and Limiting Features

## Detailed Changeset Retrieval with Error Handling

## Repository Scan with Error Handling

## Repository Scan with Error Handling